# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [76]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
schoolfilepath = "Resources/schools_complete.csv"
studentfilepath = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
schooldata = pd.read_csv(schoolfilepath)
studentdata = pd.read_csv(studentfilepath)



# Combine the data into a single dataset
combinedata = pd.merge(studentdata, schooldata, how="left", on=["school_name", "school_name"])
combinedata.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [77]:
# Total number of schools
numschool = len(combinedata["school_name"].unique())

# Total number of students
numstudent = combinedata["size"].sum()

# Total budget of all schools
totalbudget = combinedata["budget"].sum()

# Average math score
avgmathscore = combinedata["math_score"].mean()

# Average reading score
avgreadingscore = combinedata["reading_score"].mean()


# Percent of passing math scores
percentmathpass = ((combinedata["math_score"] > 70).sum()/combinedata["math_score"].count())*100

# Percent of passing reading scores
percentreadingpass = (((combinedata["reading_score"] > 70).sum()/combinedata["reading_score"].count())*100)

# Overall passing scores
overallpass = ((percentmathpass + percentreadingpass)/2)

#District Summary table
district_summary = pd.DataFrame({"Total Schools":[numschool],
                               "Total Students":[numstudent],
                               "Total Budget" : [totalbudget],
                               "Average Math Score":[avgmathscore],
                               "Average Reading Score":[avgreadingscore],
                               "% Passing Math":[percentmathpass],
                               "% Passing Reading": [percentreadingpass],
                               "% Overall Passing Rate":[overallpass]})

# Apply formatting to columns
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].map("{:.2f}%".format)

district_summary.head()




,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"130,551,930","$82,932,329,558",78.99%,81.88%,72.39%,82.97%,77.68%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [78]:
# Group student data by school
groupedStudentSchool_df = combinedata.groupby(["school_name"], as_index=False)
#groupedStudentSchool_df.head()

# Average Math score per school
sch_avgMathScore_df = pd.DataFrame(groupedStudentSchool_df["math_score"].mean())
#sch_avgMathScore_df.head()

# average Reading score per school
sch_avgReadingScore_df = pd.DataFrame(groupedStudentSchool_df["reading_score"].mean())
#sch_avgReadingScore_df.head()

#Group math scores better than 70
group_mathPass = combinedata[studentdata["math_score"]>70].groupby(["school_name"], as_index=False)
#group_mathPass.head()

# total math pass scores as a prep for the percent pass
sch_mathPass_df = pd.DataFrame(group_mathPass["math_score"].count())
sch_mathPass_df.columns= ["school_name", "#_math_pass"]
sch_mathPass_df.head()

#Group reading scores better than 70
group_readPass = studentdata[studentdata["reading_score"]>70].groupby(["school_name"], as_index=False)
#group_readPass.head()

# total reading pass scores as a prep for the percent pass
sch_readPass_df = pd.DataFrame(group_readPass["reading_score"].count())
sch_readPass_df.columns= ["school_name", "#_read_pass"]
#sch_readPass_df.head()

# Merge school table with grouped data
mergedStudentSchool_df = pd.merge(schooldata, sch_avgMathScore_df, on="school_name" )
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, sch_mathPass_df, on="school_name")
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, sch_avgReadingScore_df, on="school_name" )
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, sch_readPass_df, on="school_name" )
#mergedStudentSchool_df.head()

# Add calculated columns for math and reading percent pass
mergedStudentSchool_df["mathPassPct"] = (mergedStudentSchool_df["#_math_pass"]/mergedStudentSchool_df["size"])*100
mergedStudentSchool_df["readPassPct"] = (mergedStudentSchool_df["#_read_pass"]/mergedStudentSchool_df["size"])*100
#mergedStudentSchool_df.head()

# Add calculated column for overall pass pct
mergedStudentSchool_df["overallPassPct"] = (mergedStudentSchool_df["mathPassPct"]+mergedStudentSchool_df["readPassPct"])/2
#mergedStudentSchool_df.head()

# Add calculated Per Student Budget column
mergedStudentSchool_df["perStudentBudget"] = mergedStudentSchool_df["budget"]/mergedStudentSchool_df["size"]
#mergedStudentSchool_df.head()

# Save data unformatted for later use
school_summary_raw = pd.DataFrame(mergedStudentSchool_df[["school_name", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "mathPassPct", "readPassPct", "overallPassPct"]])
school_summary_raw.columns = ["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]
#school_summary_raw.head()

# Select and rename columns
school_summary = pd.DataFrame(mergedStudentSchool_df[["school_name", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "mathPassPct", "readPassPct", "overallPassPct"]])
school_summary.columns = ["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]
#school_summary.head()

# Apply formatting to columns
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}%".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Pass Rate"] = school_summary["% Overall Pass Rate"].map("{:.2f}%".format)

school_summary.head()



,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
0,Huang High School,District,"2,917","$1,910,635",$655.0,76.63%,81.18%,63.32%,78.81%,71.07%
1,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71%,81.16%,63.75%,78.43%,71.09%
2,Shelton High School,Charter,"1,761","$1,056,600",$600.0,83.36%,83.73%,89.89%,92.62%,91.25%
3,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29%,80.93%,64.75%,78.19%,71.47%
4,Griffin High School,Charter,"1,468","$917,500",$625.0,83.35%,83.82%,89.71%,93.39%,91.55%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [79]:
#Create sorted list of top performing schools
sort_school_summary = school_summary.sort_values("% Overall Pass Rate", ascending=False)
#sort_school_summary.head()
# Create sorted list of bottom performing schools
sort_school_summary = school_summary.sort_values("% Overall Pass Rate")
sort_school_summary.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
11,Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.84%,80.74%,64.07%,77.74%,70.91%
0,Huang High School,District,"2,917","$1,910,635",$655.0,76.63%,81.18%,63.32%,78.81%,71.07%
12,Johnson High School,District,"4,761","$3,094,650",$650.0,77.07%,80.97%,63.85%,78.28%,71.07%
1,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71%,81.16%,63.75%,78.43%,71.09%
3,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29%,80.93%,64.75%,78.19%,71.47%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [80]:
# Get data by grades
ninth_df = studentdata.loc[studentdata["grade"] == "9th"].groupby("school_name", as_index=False)
tenth_df = studentdata.loc[studentdata["grade"] == "10th"].groupby("school_name", as_index=False)
eleventh_df = studentdata.loc[studentdata["grade"] == "11th"].groupby("school_name", as_index=False)
twelfth_df = studentdata.loc[studentdata["grade"] == "12th"].groupby("school_name", as_index=False)

#Group and get average math scores
ninthM_Avg = pd.DataFrame(ninth_df["math_score"].mean())
tenthM_Avg = pd.DataFrame(tenth_df["math_score"].mean())
eleventhM_Avg = pd.DataFrame(eleventh_df["math_score"].mean())
twelfthM_Avg = pd.DataFrame(twelfth_df["math_score"].mean())

#Merge to Math score summary table
mathByGrade = pd.merge(ninthM_Avg, tenthM_Avg, on="school_name")
mathByGrade = pd.merge(mathByGrade, eleventhM_Avg, on="school_name")
mathByGrade = pd.merge(mathByGrade, twelfthM_Avg, on="school_name")
mathByGrade.columns = ["school_name","9th","10th","11th","12th"]
mathByGrade.head()



,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [81]:
#Group and get average reading scores
ninthR_Avg = pd.DataFrame(ninth_df["reading_score"].mean())
tenthR_Avg = pd.DataFrame(tenth_df["reading_score"].mean())
eleventhR_Avg = pd.DataFrame(eleventh_df["reading_score"].mean())
twelfthR_Avg = pd.DataFrame(twelfth_df["reading_score"].mean())

#Merge to Reading score summary table
readByGrade = pd.merge(ninthR_Avg, tenthR_Avg, on="school_name")
readByGrade = pd.merge(readByGrade, eleventhR_Avg, on="school_name")
readByGrade = pd.merge(readByGrade, twelfthR_Avg, on="school_name")
readByGrade.columns = ["school_name","9th","10th","11th","12th"]
readByGrade.head()

,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [83]:
# Scores by School Spending
bins = [0, 585, 615, 645, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]
scoreByBudget = school_summary_raw[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Per Student Budget"], bins=bins, labels=group_names )).mean()
scoreByBudget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Per Student Budget,,,,,
0 to 585,83.455399,83.933814,90.350436,93.325838,91.838137
585 to 615,83.599686,83.885211,90.788049,92.410786,91.599418
615 to 645,79.079225,81.891436,73.021426,83.214343,78.117884
645 to 675,76.997210,81.027843,63.972368,78.427809,71.200088


## Scores by School Size

* Perform the same operations as above, based on school size.

In [84]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [86]:
# Scores by School Size
bins = [0, 1000, 2500, 50000]
group_names = ["Small < 1000", "Medium 1000 to 2500", "Large 2500 to 5000"]
scoreBySize = school_summary_raw[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Total Students"], bins=bins, labels=group_names)).mean()
scoreBySize.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Total Students,,,,,
Small < 1000,83.821598,83.929843,91.158155,92.471895,91.815025
Medium 1000 to 2500,83.357937,83.885280,90.098249,93.246451,91.672350
Large 2500 to 5000,76.956733,80.966636,64.302528,78.324559,71.313543


## Scores by School Type

* Perform the same operations as above, based on school type.